In [1]:
# -*- coding: utf-8 -*-
# Pyomo + Gurobi: Lagrangean-based (per Karuppiah & Grossmann, 2008) – single-node loop
from dataclasses import dataclass, field
from typing import Callable, List, Dict, Tuple
import math

from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
import bisect
from pyomo.opt import SolverStatus, TerminationCondition


from typing import List, Dict

In [23]:
'''
# ========== calculate Δλ_n ==========
def lambda_diff(n: int, lambdas: List[float]) -> float:
    """
    Chained form: (λ_n - λ_{n-1}), where λ_0=0, λ_N=0, n ∈ {1,...,N}, 
    lambdas has N-1 elements
    """
    N = len(lambdas) + 1
    lam_n = lambdas[n] if 1 <= n <= N-1 else 0.0
    lam_prev = lambdas[n-1] if 2 <= n <= N-1 else 0.0
    return lam_n - lam_prev
'''

# ========== Solve Lagrangian relaxation once ==========
def solve_lagrangian_once(model_list: List[ConcreteModel], lambdas: List[float]):
    solver = SolverFactory("gurobi")
    xs, objs = [], []
    N = len(model_list)
    for n, m in enumerate(model_list):
        if hasattr(m, 'obj'):
            m.del_component(m.obj)
        lag_term = (lambdas[n+1]-lambdas[n]) * m.y
        m.obj = Objective(expr = m.obj_expr + lag_term, sense=minimize)
        solver.solve(m, tee=False)
        xs.append(value(m.y))
        objs.append(value(m.obj))
    return xs, objs, sum(objs)

# ========== subgradient lambda update ==========
def subgradient_update(
    lambdas: list[float],
    xs: list[float],                       
    zUB: float,
    zLB: float,
    alpha_fac: float = 0.5           
) -> list[float]:
    """
    g_n = x^n - x^{n+1},  n=1,...,N-1
    t = α * (zUB - zLB) / ||g||^2
    """
    N = len(xs)
    g = [xs[n] - xs[n+1] for n in range(N-1)]
    g2 = sum(gi*gi for gi in g)
    if g2 <= 1e-16:
        return lambdas[:]
    t = alpha_fac * max(0.0, zUB - zLB) / g2
    new_lambdas = [0]
    for i, lam in enumerate(lambdas):
        if 1<=i<=(N-1):
            new_lambdas.append(lam + t * g[i-1]) 
    new_lambdas.append(0)
    return new_lambdas

# ========== main loop ==========
def lagrangian_loop(model_list: List[ConcreteModel],
                    max_iter: int = 20,
                    zUB: float = 1e9,
                    t: float = 0.5,
                    lam0: List[float] = None):
    N = len(model_list)
    lambdas = lam0 if lam0 is not None else [0.0]*(N+1)
    history = []
    for k in range(max_iter):
        xs, objs, zLB = solve_lagrangian_once(model_list, lambdas)
        history.append((k, zLB, xs, lambdas[:]))
        print(f"iter {k:02d}: zLB={zLB:.6f}, xs={['%.4f'%x for x in xs]}, λ={['%.3f'%l for l in lambdas]}")
        print('lower bound is ',sum(objs))
        lambdas = subgradient_update(lambdas, xs, zUB, zLB)
    return history

In [30]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return -(y-1)**2+1

v_list = [v_1, v_2]

# build model for each scenario 
m1 = ConcreteModel()
m1.y = Var(bounds=(0, 1))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y = Var(bounds=(0, 1))
m2.x2 = Var()
m2.c1 = Constraint(expr=m2.x2 == -(m2.y-1)**2)
m2.obj_expr = Expression(expr=m2.x2+1)

model_list = [m1, m2]
# lower bound and upper bound
ylb = 0
yub = 0.2

lam0 = [0,0.5, 0]
lagrangian_loop(model_list, max_iter=15, zUB=-0.1, t=0.3, lam0=lam0)

iter 00: zLB=-0.500000, xs=['1.0000', '0.0000'], λ=['0.000', '0.500', '0.000']
lower bound is  -0.5
iter 01: zLB=-0.357143, xs=['0.5102', '0.0000'], λ=['0.000', '0.700', '0.000']
lower bound is  -0.35714285714285715
iter 02: zLB=-0.262605, xs=['0.2758', '0.0000'], λ=['0.000', '0.952', '0.000']
lower bound is  -0.26260504201680673
iter 03: zLB=-0.447262, xs=['0.1608', '1.0000'], λ=['0.000', '1.247', '0.000']
lower bound is  -0.44726229256017613
iter 04: zLB=-0.280253, xs=['0.2312', '1.0000'], λ=['0.000', '1.040', '0.000']
lower bound is  -0.2802531669622974
iter 05: zLB=-0.270974, xs=['0.2937', '0.0000'], λ=['0.000', '0.923', '0.000']
lower bound is  -0.27097435801435493
iter 06: zLB=-0.419647, xs=['0.1697', '1.0000'], λ=['0.000', '1.214', '0.000']
lower bound is  -0.4196467651955715
iter 07: zLB=-0.265982, xs=['0.2397', '1.0000'], λ=['0.000', '1.021', '0.000']
lower bound is  -0.26598209064603906
iter 08: zLB=-0.274122, xs=['0.3006', '0.0000'], λ=['0.000', '0.912', '0.000']
lower bound

[(0, -0.5, [1.0, 0.0], [0, 0.5, 0]),
 (1, -0.35714285714285715, [0.5102040816326531, 0.0], [0, 0.7, 0]),
 (2, -0.26260504201680673, [0.27584563237059534, 0.0], [0, 0.952, 0]),
 (3, -0.44726229256017613, [0.16083804259958795, 1.0], [0, 1.2467392, 0]),
 (4,
  -0.2802531669622974,
  [0.23121506945097245, 1.0],
  [0, 1.0398290264049417, 0]),
 (5,
  -0.27097435801435493,
  [0.2937084108051672, 0.0],
  [0, 0.922596521058115, 0]),
 (6,
  -0.4196467651955715,
  [0.16972562816812592, 1.0],
  [0, 1.213657913453465, 0]),
 (7,
  -0.26598209064603906,
  [0.23974505143493363, 1.0],
  [0, 1.0211632538325017, 0]),
 (8,
  -0.2741224675613389,
  [0.3005725088876692, 0.0],
  [0, 0.9120011293640455, 0]),
 (9,
  -0.40969932066451287,
  [0.17313394822363215, 1.0],
  [0, 1.2016524820538095, 0]),
 (10,
  -0.2608355947484656,
  [0.24296240046133932, 1.0],
  [0, 1.014379513234264, 0]),
 (11,
  -0.27528414376523325,
  [0.3031254392342304, 0.0],
  [0, 0.9081525604075622, 0]),
 (12,
  -0.4060871451962709,
  [0.174

# P1

In [ ]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return np.sqrt(np.abs(y))

v_list = [v_1, v_2]

# lower bound and upper bound
ylb = -0.2
yub = 0.2
bounds = [ylb, yub]
# build model for each scenario 
m1 = ConcreteModel()
m1.y = Var(bounds=(ylb, yub))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y = Var(bounds=(ylb, yub))
m2.x2 = Var(bounds=(0, 1))
m2.c1 = Constraint(expr=m2.x2**2 >= m2.y)
m2.c2 = Constraint(expr=m2.x2**2 >= -m2.y)
m2.obj_expr = Expression(expr=m2.x2)

model_list = [m1, m2]
lam0 = [0,0.5, 0]
lagrangian_loop(model_list, max_iter=15, zUB=-0.1, t=0.3, lam0=lam0)

In [33]:
# build function for each scenario
def v_1(y):  
    return - np.sqrt(np.abs(y))

def v_2(y): 
    return -(y-1)**2+1

v_list = [v_1, v_2]

# build model for each scenario 
m1 = ConcreteModel()
m1.y = Var(bounds=(0, 1))
m1.x1 = Var(bounds=(0, 1))
m1.x11 = Var(bounds=(0, None))
m1.c1 = Constraint(expr=m1.x11**2 == m1.y**2)
m1.c2 = Constraint(expr=m1.x11 == (m1.x1)**2)
m1.obj_expr = Expression(expr=-m1.x1)

m2 = ConcreteModel()
m2.y = Var(bounds=(0, 1))
m2.x2 = Var()
m2.c1 = Constraint(expr=m2.x2 == -(m2.y-1)**2)
m2.obj_expr = Expression(expr=m2.x2+1)

model_list = [m1, m2]
# lower bound and upper bound
ylb = 0
yub = 0.2

lam0 = [0, 0.5, 0]
lagrangian_loop(model_list, max_iter=15, zUB=0.0, t=0.3, lam0=lam0)

iter 00: zLB=-0.500000, xs=['1.0000', '0.0000'], λ=['0.000', '0.500', '0.000']
lower bound is  -0.5
iter 01: zLB=-0.333333, xs=['0.4444', '0.0000'], λ=['0.000', '0.750', '0.000']
lower bound is  -0.3333333333333333
iter 02: zLB=-0.347222, xs=['0.1975', '1.0000'], λ=['0.000', '1.125', '0.000']
lower bound is  -0.3472222222222222
iter 03: zLB=-0.275132, xs=['0.3028', '0.0000'], λ=['0.000', '0.909', '0.000']
lower bound is  -0.2751322751322752
iter 04: zLB=-0.546402, xs=['0.1346', '1.0000'], λ=['0.000', '1.363', '0.000']
lower bound is  -0.5464022859856195
iter 05: zLB=-0.286007, xs=['0.2279', '1.0000'], λ=['0.000', '1.047', '0.000']
lower bound is  -0.2860066187387981
iter 06: zLB=-0.289998, xs=['0.3364', '0.0000'], λ=['0.000', '0.862', '0.000']
lower bound is  -0.28999752679169394
iter 07: zLB=-0.486446, xs=['0.1495', '1.0000'], λ=['0.000', '1.293', '0.000']
lower bound is  -0.48644616087767045
iter 08: zLB=-0.255364, xs=['0.2465', '1.0000'], λ=['0.000', '1.007', '0.000']
lower bound is

[(0, -0.5, [1.0, 0.0], [0, 0.5, 0]),
 (1, -0.3333333333333333, [0.4444444444444444, 0.0], [0, 0.75, 0]),
 (2, -0.3472222222222222, [0.19753086419753085, 1.0], [0, 1.125, 0]),
 (3,
  -0.2751322751322752,
  [0.3027910752778477, 0.0],
  [0, 0.9086538461538461, 0]),
 (4,
  -0.5464022859856195,
  [0.134573811234599, 1.0],
  [0, 1.3629807692307694, 0]),
 (5,
  -0.2860066187387981,
  [0.22792944106486027, 1.0],
  [0, 1.047296836228288, 0]),
 (6,
  -0.28999752679169394,
  [0.336394262181197, 0.0],
  [0, 0.862076317566583, 0]),
 (7,
  -0.48644616087767045,
  [0.14950856096942086, 1.0],
  [0, 1.2931144763498745, 0]),
 (8,
  -0.25536390101048834,
  [0.24647031198009542, 1.0],
  [0, 1.007135018736439, 0]),
 (9,
  -0.2984398034738258,
  [0.3562652651899831, 0.0],
  [0, 0.8376898694142381, 0]),
 (10,
  -0.4554946731039076,
  [0.1583401178622147, 1.0],
  [0, 1.256534804121357, 0]),
 (11,
  -0.25356468125761367,
  [0.25718019032510087, 0.0],
  [0, 0.9859417280043348, 0]),
 (12,
  -0.6479557128449114,
